In [1]:
# packages you may need to import
from sklearn.datasets import load_files
from scipy.stats import uniform, randint as sp_randint
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import KFold,StratifiedKFold,RandomizedSearchCV, GridSearchCV
import nltk
from nltk.cluster import KMeansClusterer, cosine_distance
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
import string
from sklearn.pipeline import make_pipeline, Pipeline

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Declare the categories
categories = ['neg', 'pos']

# Load the dataset
docs_to_train = load_files("/Users/MSD/OneDrive - Dartmouth College/189/Assg4/aclImdb/train", description=None, categories=categories, load_content=True, shuffle=True, encoding='utf-8', decode_error='strict', random_state=0)

In [3]:
y_raw = docs_to_train.target

In [4]:
docs_to_train.target_names #0 is negative and 1 is positive

['neg', 'pos']

In [5]:
text_raw = docs_to_train.data

In [6]:
print("The data type is", type(text_raw), "and the length of the raw text data is,", len(text_raw))

The data type is <class 'list'> and the length of the raw text data is, 25000


Justification for preprocessing: 

I remove punctuation within my document. I then remove anything other than an alphabet/number. Since I remove punctuation and tokenized my data, I remove words smaller than 3 characters to remove the "noise" from the document. 



In [7]:
#Removing stopwords and other characters as defined by me
from nltk.stem import WordNetLemmatizer
text_ready = []


#stopw = stopwords.words('english') + list(string.punctuation)  +["’","”","—","“"] 
stopw = list(string.punctuation)  +["’","”","—","“"] 

for i in range(0,len(text_raw)):
    x = text_raw[i]
    text = re.sub('[^a-zA-Z0-9]', ' ', x)
    
   
    #Words
    tokens = [stop for stop in word_tokenize(text.lower()) if stop not in stopw] #removes stopwords, punctuation and custom filter
    tokens = [w for w in tokens if re.findall(r"(\w{3})",w) ] #removes anything less than 3 characters
    review = ' '.join(tokens)
    text_ready.append(review)
    


In [8]:
text_raw[1]

'Words can\'t describe how bad this movie is. I can\'t explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There are so many clichés, mistakes (and all other negative things you can imagine) here that will just make you cry. To start with the technical first, there are a LOT of mistakes regarding the airplane. I won\'t list them here, but just mention the coloring of the plane. They didn\'t even manage to show an airliner in the colors of a fictional airline, but instead used a 747 painted in the original Boeing livery. Very bad. The plot is stupid and has been done many times before, only much, much better. There are so many ridiculous moments here that i lost count of it really early. Also, I was on the bad guys\' side all the time in the movie, because the good guys were so stupid. "Executive Decision" should without a doubt be you\'re choice over this one, even the "Turbulence"-movie

In [9]:
text_ready[1]

'words can describe how bad this movie can explain writing only you have too see for yourself get grip how horrible movie really can not that recommend you that there are many clich mistakes and all other negative things you can imagine here that will just make you cry start with the technical first there are lot mistakes regarding the airplane won list them here but just mention the coloring the plane they didn even manage show airliner the colors fictional airline but instead used 747 painted the original boeing livery very bad the plot stupid and has been done many times before only much much better there are many ridiculous moments here that lost count really early also was the bad guys side all the time the movie because the good guys were stupid executive decision should without doubt you choice over this one even the turbulence movies are better fact every other movie the world better than this one'

In [10]:
X_train_text,X_test_text,y_train,y_test=train_test_split(text_ready,
                                              y_raw ,
                                              test_size=0.1,
                                                train_size = 0.4,
                                               stratify = y_raw,
                                              random_state=0)

In [11]:
print("Number of positives reviews", len(y_train[y_train == 1])) #pos reviews


Number of positives reviews 5000


In [12]:
print("Number of negative reviews", len(y_train[y_train ==0])) #neg reviews

Number of negative reviews 5000


Justification for CountVectorizer:

In my opinion, I think we should ignore 1 grams and consider batch of words that might be the decider in whether a review is positive to negative. Phrases like "really good", "really bad". I will find out whether this is valid. I limit my features to 50,000. 

In [13]:
# write your code here

vect=CountVectorizer(ngram_range = (2,3)).fit(X_train_text)
X_train = vect.transform(X_train_text)
X_test = vect.transform(X_test_text)

In [14]:
print("The number of rows are, ", X_train.shape[0])

The number of rows are,  10000


In [15]:
print("The number of features are, ", X_train.shape[1])

The number of features are,  2254415


In [16]:
vect.get_feature_names()[:20]

['000 000',
 '000 000 000',
 '000 000 and',
 '000 000 could',
 '000 000 give',
 '000 000 hollow',
 '000 000 marries',
 '000 000 mega',
 '000 000 prison',
 '000 000 retitled',
 '000 000 stupid',
 '000 000 the',
 '000 000 times',
 '000 250',
 '000 250 000',
 '000 advance',
 '000 advance goes',
 '000 after',
 '000 after cohn',
 '000 and']

### Vanilla Model

In [18]:
# write your code here
model=LogisticRegression(C=0.001)
k_fold = StratifiedKFold(5,random_state = 9)
scores = cross_val_score(model,X_train,y_train,cv =k_fold,scoring = "accuracy")
print('The mean accuracy score from our 5 fold cross validation is, ', round(scores.mean()*100,2) ,'% from LR')

C:\Users\MSD\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The mean accuracy score from our 5 fold cross validation is,  75.77 % from LR


In [19]:
lr = [("lr",LogisticRegression())]  #Feature Scaling and SVM classifier
lr1 = Pipeline(lr)  #Pipeline
param_lr1 = [{ "lr__C":[0.001, 0.01, 0.1, 1, 10]}]
grid_lr = GridSearchCV(lr1,param_grid = param_lr1,cv = 5,scoring = "accuracy",verbose = 3, n_jobs = -1) #GridSearch and parameter grid


In [20]:
grid_lr.fit(X_train,y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  11 out of  25 | elapsed:   35.4s remaining:   45.1s
[Parallel(n_jobs=-1)]: Done  20 out of  25 | elapsed:  1.2min remaining:   17.8s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:  1.3min finished
C:\Users\MSD\AppData\Local\conda\conda\envs\tensorflow\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lr', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'lr__C': [0.001, 0.01, 0.1, 1, 10]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=3)

In [21]:
print('The mean accuracy score from our 5 fold cross validation is, ', round(grid_lr.best_score_*100,2) ,'% from LR')

The mean accuracy score from our 5 fold cross validation is,  82.99 % from LR


In [22]:
grid_lr.best_params_

{'lr__C': 10}

In [23]:
pred2 = grid_lr.predict(X_test)


In [24]:
print('The accuracy is, ', round(accuracy_score(y_test,pred2)*100,2) ,'% from LR')
print('The precision is, ', round(precision_score(y_test,pred2)*100,2) ,'% from LR')
print('The recall is, ', round(recall_score(y_test,pred2)*100,2) ,'% from LR')
print('The F1 score is, ', round(f1_score(y_test,pred2)*100,2) ,'% from LR')


The accuracy is,  84.84 % from LR
The precision is,  83.84 % from LR
The recall is,  86.32 % from LR
The F1 score is,  85.06 % from LR


In [25]:
confusion_matrix(y_test,pred2)

array([[1042,  208],
       [ 171, 1079]], dtype=int64)

### Optimizing the Model

Justification:

After hyperparameterizing y Logisitic Regression model, the results were



The accuracy is,  84.84 % from LR
The precision is,  83.84 % from LR
The recall is,  86.32 % from LR
The F1 score is,  85.06 % from LR

Understandably, my assumption about ignoring 1 grams was misplaced. This may be due to people not typing the same way so not all phrases are common across reviews. Additionally, I will have to remove numbers as they also count as noise and take away from the intrepretability of the model. Additionally, I will incorporate stop words that will help me get to the core of each review and determine its sentiment. I will also Stem the words using snowball stemmer to help me arrive at the core of a word which generally should help. I also noticed a lot of noisy words like "aaa" "bbb" which don't add anything to the model. I will remove words with 3 or more duplicate characters. This will not affect words like "good" but will remove the terms like "trashhhh". 

After these preprocessing changes, I will run it through a TFID vectorizer with ngram(1,5) and consider words more than a frequency of 10 or more. This vector result will be put down a Deep Learning Neural Network model. The parameters used in that model is out of scope for this assignment and will not be explained further. 

In [26]:
#Removing stopwords and other characters as defined by me
from nltk.stem import WordNetLemmatizer
text_ready = []
from nltk.stem.snowball import SnowballStemmer

stem = SnowballStemmer("english")

stopw = stopwords.words('english') + list(string.punctuation)  +["’","”","—","“"]


for i in range(0,len(text_raw)):
    x = text_raw[i]
    text = re.sub(r'((.)\2{2,})', ' ', x) #remove three or more consecutive characters like 'trashhhh'
    text = re.sub('[^a-zA-Z]', ' ', text)
    
   
    #Words
    tokens = [stop for stop in word_tokenize(text.lower()) if stop not in stopw] #removes stopwords, punctuation and custom filter
    #tokens = [w for w in tokens if w.isalpha()]  #remove every that is not fully alphabetic
    tokens = [w for w in tokens if re.findall(r"(\w{3})",w) ] #removes anything less than 3 characters
    tokens = [stem.stem(w) for w in tokens]
    review = ' '.join(tokens)
    text_ready.append(review)
    


In [27]:
X_train_text,X_test_text,y_train,y_test=train_test_split(text_ready,
                                              y_raw ,
                                              test_size=0.1,
                                                train_size = 0.4,
                                               stratify = y_raw,
                                              random_state=0)

In [28]:

# write your code here
 # vectorise & transform
vect = TfidfVectorizer(min_df=10,ngram_range=(1,5)).fit(X_train_text)
X_train_vectorized = vect.transform(X_train_text)
X_test_vectorized = vect.transform(X_test_text)

 

In [29]:
from keras.models import Sequential 
from keras.layers import Dense, Embedding, Convolution1D, GlobalMaxPooling1D
from keras import layers, models, callbacks, optimizers

model = models.Sequential()
 
model.add(layers.Dense(50, activation='relu', input_shape=(16322,)))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer='RMSProp',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(X_train_vectorized, y_train, epochs=2, batch_size=250)

scores = model.evaluate(X_test_vectorized, y_test)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Using TensorFlow backend.


Epoch 1/2
10000/10000 [==============================] - 2s 172us/step - loss: 0.5573 - acc: 0.7746
Epoch 2/2
2500/2500 [==============================] - 0s 182us/step

acc: 89.44%


In [30]:
pred_nn = model.predict(X_test_vectorized)
pred_nn = (pred_nn >= 0.5)
print('The accuracy is, ', round(accuracy_score(y_test,pred_nn)*100,2) ,'% from the Neural Network')
print('The precision is, ', round(precision_score(y_test,pred_nn)*100,2) ,'% from the Neural Network')
print('The recall is, ', round(recall_score(y_test,pred_nn)*100,2) ,'% from the Neural Network')
print('The F1 score is, ', round(f1_score(y_test,pred_nn)*100,2) ,'% from the Neural Network')


The accuracy is,  89.44 % from the Neural Network
The precision is,  89.89 % from the Neural Network
The recall is,  88.88 % from the Neural Network
The F1 score is,  89.38 % from the Neural Network


As we can see the results are greatly improved. Further improvement can be made by considering an Embedding layer and Convolution1D and MaxPooling parameters. 

###  Latent Dirichlet Allocation Simple Topic Modelling

In [31]:
def print_LDA_results(lda_model, feature_names, n_top_words=15):
    for topic_idx, topic in enumerate(lda_model.components_):
        message = "Topic %d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
        print()

In [32]:
# write your code here
vect2=CountVectorizer(max_df = 0.15, max_features = 10000)
X_train2 = vect2.fit_transform(X_train_text)
X_test2 = vect2.transform(X_test_text)

In [33]:
feature_names = vect2.get_feature_names()

In [34]:
lda_model = LatentDirichletAllocation(n_components = 10, learning_method = 'batch', max_iter = 30, random_state = 9).fit(X_train2)

In [35]:
print_LDA_results(lda_model, feature_names, n_top_words=15)

Topic 0: famili son father role oscar nomin mother award live boy indian day american georg real

Topic 1: comedi origin music funni cast version enjoy anim role fun song book wonder star fan

Topic 2: game jane role western jame star cast stewart novel robert john real seri jim bond

Topic 3: murder kill polic wife cop killer john role car turn crime michael father town harri

Topic 4: beauti world live point differ real may emot human audienc without viewer view fact quit

Topic 5: war action world fight american star soldier human new hero effect set earth sci seri

Topic 6: episod girl young woman mother music season seri husband danc old daughter women famili new

Topic 7: guy noth worst minut funni got wast laugh go bore thought stupid line aw everi

Topic 8: paul role cast page betti smith young ann che mari job henri jeff harri friend

Topic 9: horror kill effect zombi hous gore killer blood night pretti dead budget girl scari set



The output names seem pretty good. Some of the results provide very little clue towards the title of the review topic, as it could apply to a number of topics. For Topic 5, I would guess a war/action film that features sci-fi, topic 9 as your run of the mill horror film featuring zombies, topic 2 is a western setting topic/movie, topic 2 is refers to an animated movie topic. 

Note: I used the data from my improved model with the new pre-processing. 